# LoRa Communication Data Analysis

**Roboter Gruppe 9 - Interactive Data Analysis Notebook**

This notebook provides interactive analysis of LoRa communication data collected from the ESP32 devices.

## Setup

Required packages:
```bash
pip install pandas matplotlib seaborn sqlite3 jupyter
```

## 1. Import Libraries

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Libraries imported successfully")

## 2. Load Data from SQLite Database

In [ ]:
# Path to your SQLite database
DB_PATH = 'lora_data.db'  # Change this to your database path

# Connect to database
conn = sqlite3.connect(DB_PATH)

# Load data
query = """
SELECT 
    timestamp,
    esp_timestamp,
    role,
    rssi,
    snr,
    sequence,
    message_count,
    connection_state,
    packet_loss,
    led_state,
    touch_state
FROM lora_messages
ORDER BY timestamp
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"✓ Loaded {len(df)} data points")
print(f"✓ Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"✓ Duration: {(df['timestamp'].max() - df['timestamp'].min()).total_seconds() / 3600:.1f} hours")

# Display first few rows
df.head(10)

## 3. Quick Statistics

In [ ]:
print("=" * 60)
print("QUICK STATISTICS")
print("=" * 60)

print(f"\n📦 Data Points:")
print(f"   Total messages: {len(df)}")
print(f"   Role: {df['role'].iloc[0] if len(df) > 0 else 'N/A'}")

print(f"\n📡 RSSI Statistics:")
print(f"   Mean:   {df['rssi'].mean():.1f} dBm")
print(f"   Median: {df['rssi'].median():.1f} dBm")
print(f"   Min:    {df['rssi'].min()} dBm")
print(f"   Max:    {df['rssi'].max()} dBm")
print(f"   Std:    {df['rssi'].std():.1f} dBm")

print(f"\n📊 SNR Statistics:")
print(f"   Mean:   {df['snr'].mean():.1f} dB")
print(f"   Median: {df['snr'].median():.1f} dB")
print(f"   Min:    {df['snr'].min()} dB")
print(f"   Max:    {df['snr'].max()} dB")

print(f"\n📉 Packet Loss:")
print(f"   Mean:   {df['packet_loss'].mean():.2f}%")
print(f"   Median: {df['packet_loss'].median():.2f}%")
print(f"   Max:    {df['packet_loss'].max():.2f}%")
print(f"   Final:  {df['packet_loss'].iloc[-1]:.2f}%")

print(f"\n🔗 Connection States:")
for state, count in df['connection_state'].value_counts().items():
    print(f"   {state:10s}: {count:4d} ({count/len(df)*100:5.1f}%)")

print("\n" + "="*60)

## 4. RSSI Analysis

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('RSSI Signal Strength Analysis', fontsize=16, fontweight='bold')

# 1. RSSI over time
ax1 = axes[0, 0]
ax1.plot(df['timestamp'], df['rssi'], linewidth=1, alpha=0.7)
ax1.axhline(y=-80, color='g', linestyle='--', label='Excellent (-80 dBm)')
ax1.axhline(y=-100, color='orange', linestyle='--', label='Weak (-100 dBm)')
ax1.set_xlabel('Time')
ax1.set_ylabel('RSSI (dBm)')
ax1.set_title('RSSI Timeline')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. RSSI histogram
ax2 = axes[0, 1]
ax2.hist(df['rssi'], bins=30, edgecolor='black', alpha=0.7)
ax2.axvline(df['rssi'].mean(), color='red', linestyle='--', label=f"Mean: {df['rssi'].mean():.1f} dBm")
ax2.axvline(df['rssi'].median(), color='blue', linestyle='--', label=f"Median: {df['rssi'].median():.1f} dBm")
ax2.set_xlabel('RSSI (dBm)')
ax2.set_ylabel('Frequency')
ax2.set_title('RSSI Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# 3. RSSI quality zones
ax3 = axes[1, 0]
excellent = len(df[df['rssi'] >= -80])
good = len(df[(df['rssi'] >= -100) & (df['rssi'] < -80)])
weak = len(df[df['rssi'] < -100])
labels = ['Excellent\n(≥-80 dBm)', 'Good\n(-100 to -80)', 'Weak\n(<-100 dBm)']
sizes = [excellent, good, weak]
colors = ['#2ecc71', '#f39c12', '#e74c3c']
ax3.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax3.set_title('Signal Quality Distribution')

# 4. RSSI moving average
ax4 = axes[1, 1]
df['rssi_ma'] = df['rssi'].rolling(window=20).mean()
ax4.plot(df['timestamp'], df['rssi'], alpha=0.3, label='Raw RSSI')
ax4.plot(df['timestamp'], df['rssi_ma'], linewidth=2, label='20-point Moving Average')
ax4.set_xlabel('Time')
ax4.set_ylabel('RSSI (dBm)')
ax4.set_title('RSSI with Moving Average')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. SNR Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Signal-to-Noise Ratio (SNR) Analysis', fontsize=16, fontweight='bold')

# SNR over time
axes[0].plot(df['timestamp'], df['snr'], linewidth=1, alpha=0.7, color='green')
axes[0].axhline(y=0, color='red', linestyle='--', label='SNR = 0 dB')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('SNR (dB)')
axes[0].set_title('SNR Timeline')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# SNR histogram
axes[1].hist(df['snr'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(df['snr'].mean(), color='red', linestyle='--', label=f"Mean: {df['snr'].mean():.1f} dB")
axes[1].set_xlabel('SNR (dB)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('SNR Distribution')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. RSSI vs SNR Correlation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Scatter plot with color based on connection state
state_colors = {'UNKNOWN': 'gray', 'CONNECT': 'blue', 'OK': 'green', 'WEAK': 'orange', 'LOST': 'red'}

for state in df['connection_state'].unique():
    mask = df['connection_state'] == state
    ax.scatter(df[mask]['rssi'], df[mask]['snr'], 
              alpha=0.5, s=20, 
              c=state_colors.get(state, 'gray'), 
              label=state)

# Calculate correlation
correlation = df['rssi'].corr(df['snr'])

ax.set_xlabel('RSSI (dBm)', fontsize=12)
ax.set_ylabel('SNR (dB)', fontsize=12)
ax.set_title(f'RSSI vs SNR Correlation (r = {correlation:.3f})', fontsize=14, fontweight='bold')
ax.legend(title='Connection State')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.3f}")
if correlation > 0.7:
    print("✓ Strong positive correlation")
elif correlation > 0.4:
    print("✓ Moderate positive correlation")
else:
    print("⚠ Weak correlation")

## 7. Packet Loss Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Packet Loss Analysis', fontsize=16, fontweight='bold')

# Packet loss over time
axes[0].plot(df['timestamp'], df['packet_loss'], linewidth=2, color='red', alpha=0.7)
axes[0].fill_between(df['timestamp'], df['packet_loss'], alpha=0.3, color='red')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Packet Loss (%)')
axes[0].set_title('Packet Loss Timeline')
axes[0].grid(True, alpha=0.3)

# Packet loss vs RSSI
axes[1].scatter(df['rssi'], df['packet_loss'], alpha=0.5, s=20)
axes[1].set_xlabel('RSSI (dBm)')
axes[1].set_ylabel('Packet Loss (%)')
axes[1].set_title('Packet Loss vs Signal Strength')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find periods with high packet loss
high_loss = df[df['packet_loss'] > 5.0]
print(f"\n⚠ Periods with >5% packet loss: {len(high_loss)} data points")
if len(high_loss) > 0:
    print(f"   Average RSSI during high loss: {high_loss['rssi'].mean():.1f} dBm")
    print(f"   Average SNR during high loss: {high_loss['snr'].mean():.1f} dB")

## 8. Connection State Timeline

In [ ]:
# Map states to numeric values
state_map = {'UNKNOWN': 0, 'CONNECT': 1, 'OK': 2, 'WEAK': 1.5, 'LOST': 0}
df['state_numeric'] = df['connection_state'].map(state_map)

fig, ax = plt.subplots(figsize=(16, 6))

# Plot connection state
state_colors = {'UNKNOWN': '#95a5a6', 'CONNECT': '#3498db', 'OK': '#2ecc71', 
               'WEAK': '#f39c12', 'LOST': '#e74c3c'}

for state in df['connection_state'].unique():
    mask = df['connection_state'] == state
    ax.fill_between(df[mask]['timestamp'], 0, df[mask]['state_numeric'], 
                   color=state_colors.get(state, 'gray'), alpha=0.6, label=state)

ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Connection Quality', fontsize=12)
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['Lost', 'Weak', 'OK'])
ax.set_title('Connection State Timeline', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', ncol=5)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 9. Hourly Statistics

In [ ]:
# Add hour column
df['hour'] = df['timestamp'].dt.hour

# Calculate hourly statistics
hourly_stats = df.groupby('hour').agg({
    'rssi': ['mean', 'min', 'max', 'std'],
    'snr': ['mean', 'min', 'max'],
    'packet_loss': ['mean', 'max'],
    'timestamp': 'count'
}).round(2)

print("\n📊 HOURLY STATISTICS")
print("="*80)
print(hourly_stats)

# Plot hourly averages
fig, axes = plt.subplots(2, 1, figsize=(14, 8))
fig.suptitle('Hourly Statistics', fontsize=16, fontweight='bold')

# Hourly RSSI
hourly_rssi = df.groupby('hour')['rssi'].mean()
axes[0].bar(hourly_rssi.index, hourly_rssi.values, alpha=0.7, color='blue')
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Average RSSI (dBm)')
axes[0].set_title('Average RSSI by Hour')
axes[0].grid(True, alpha=0.3, axis='y')

# Hourly packet loss
hourly_loss = df.groupby('hour')['packet_loss'].mean()
axes[1].bar(hourly_loss.index, hourly_loss.values, alpha=0.7, color='red')
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Average Packet Loss (%)')
axes[1].set_title('Average Packet Loss by Hour')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 10. Export Results

In [ ]:
# Export to CSV
output_csv = 'lora_analysis_results.csv'
df.to_csv(output_csv, index=False)
print(f"✓ Data exported to: {output_csv}")

# Export summary statistics
summary = df.describe()
summary_csv = 'lora_summary_statistics.csv'
summary.to_csv(summary_csv)
print(f"✓ Summary statistics exported to: {summary_csv}")

print(f"\n✓ Analysis complete!")
print(f"  Total data points analyzed: {len(df)}")
print(f"  Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")

## 11. Custom Analysis Section

Use this section for your own custom analysis and experiments.

In [ ]:
# Your custom analysis code here
# Example: Find the best and worst 5-minute windows

df['rssi_5min_avg'] = df['rssi'].rolling(window=150).mean()  # ~150 samples = 5 min at 2s interval

best_window_idx = df['rssi_5min_avg'].idxmax()
worst_window_idx = df['rssi_5min_avg'].idxmin()

print(f"\n🏆 Best 5-minute window:")
print(f"   Time: {df.loc[best_window_idx, 'timestamp']}")
print(f"   Average RSSI: {df.loc[best_window_idx, 'rssi_5min_avg']:.1f} dBm")

print(f"\n⚠ Worst 5-minute window:")
print(f"   Time: {df.loc[worst_window_idx, 'timestamp']}")
print(f"   Average RSSI: {df.loc[worst_window_idx, 'rssi_5min_avg']:.1f} dBm")